In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Route_11_bus_51938440.txt',encoding='utf-8')

In [3]:
df.shape

(142324, 33)

In [4]:
df['ACTDATETIME'] = df['ACTDATETIME'].astype('datetime64[ns]')

In [5]:
df.head()

,BUSRDID,DATATYPE,ROUTEID,SUBROUTEID,PRODUCTID,STATIONSEQNUM,STATIONNUM,ISARRLFT,ACTDATETIME,RECDATETIME,...,ONPNUM,OFFPNUM,LEFTPNUM,TPTDATA,DOORSTATE,PECCANCYTYPE,PECCANCYTIME,STANDARDVALUE,PECCANCYVALUE,GPSMILE
0,28408848073,3,11,11,51938440,NaN,0,NaN,2017-05-21 06:43:01,5/21/2017 6:45:21 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,1/1/1970 8:00:00 AM,NaN,NaN,31380518
1,28408848397,3,11,11,51938440,NaN,0,NaN,2017-05-21 06:43:10,5/21/2017 6:45:23 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,1/1/1970 8:00:00 AM,NaN,NaN,31380518
2,28408848933,3,11,11,51938440,NaN,0,NaN,2017-05-21 06:43:21,5/21/2017 6:45:26 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,1/1/1970 8:00:00 AM,NaN,NaN,31380518
3,28408849287,3,11,11,51938440,NaN,0,NaN,2017-05-21 06:43:31,5/21/2017 6:45:28 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,1/1/1970 8:00:00 AM,NaN,NaN,31380518
4,28408849608,3,11,11,51938440,NaN,0,NaN,2017-05-21 06:43:40,5/21/2017 6:45:30 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,1/1/1970 8:00:00 AM,NaN,NaN,31380518


In [6]:
#得把运营类型在2，3删除，就是在首末站的那些数据
db = df[df['BUSSID'].isin([4,5])]

### Data Check
选择工作日的数据，并输出查看，寻找noise filter的方式。
通过查看有很多在首末站的重复记录，按照Nonlinear Dynamics课程的思路，创建一个low frequency filter，只让低频信号通过。
高频就是terminal station的GPS重复出现高达50(自定义)次以上，可以认为是high frequency即noise。

In [7]:
db_1 = db[(db['ACTDATETIME'] >'2017-05-22 00:00:00') & (db['ACTDATETIME'] < '2017-05-27 00:00:00')]

In [8]:
db_2 = db_1.loc[:,['ACTDATETIME','LONGITUDE','LATITUDE','BUSSID','STATIONTYPE','STATIONSEQNUM']]

In [9]:
db_2.to_csv('check.csv',index = False)

## Noise filter 

In [85]:
#LonLat = db_2.loc[:,['LONGITUDE','LATITUDE']].round(6)
LonLat = db.loc[:,['LONGITUDE','LATITUDE']].round(6)

### Noise filter by columns

In [71]:
a = LonLat.iloc[:,0]
b = LonLat.iloc[:,1]
a_1 = a.groupby((a.shift() != a).cumsum())\
.filter(lambda x: len(x) <30)
b_1 = b.groupby((b.shift() != a).cumsum())\
.filter(lambda x: len(x) <30)

In [73]:
a_1.dropna(inplace=True)

In [58]:
b_1.dropna(inplace=True)

In [42]:
a_1.to_csv('before_DCE_for_route11_Bus51938440.txt',header=None, index=None, sep=' ', mode='w')

### Noise filter by Table with two operating direction
It's better to filter noise on the whole table, because filtering noise by comlumns may end up having columns with different length.

A side note: One still have to do filtering based on only one column, here I decided to do that by the longitude column.

In [7]:
bbb = db.loc[:,['ACTDATETIME','LONGITUDE','LATITUDE','BUSSID','ISARRLFT','STATIONTYPE','STATIONSEQNUM']].copy()

In [8]:
bbb

,ACTDATETIME,LONGITUDE,LATITUDE,BUSSID,ISARRLFT,STATIONTYPE,STATIONSEQNUM
114,2017-05-21 07:02:04,119.987885,31.849114,4,1.0,6.0,1.0
115,2017-05-21 07:02:08,119.987814,31.849121,4,NaN,NaN,NaN
116,2017-05-21 07:02:17,119.987555,31.848926,4,NaN,NaN,NaN
117,2017-05-21 07:02:18,119.987536,31.848903,4,2.0,6.0,1.0
118,2017-05-21 07:02:28,119.987140,31.848926,4,NaN,NaN,NaN
119,2017-05-21 07:02:37,119.986413,31.849248,4,NaN,NaN,NaN
120,2017-05-21 07:02:47,119.985746,31.849065,4,NaN,NaN,NaN
121,2017-05-21 07:02:58,119.985229,31.848340,4,NaN,NaN,NaN
122,2017-05-21 07:03:08,119.984545,31.847718,4,NaN,NaN,NaN
123,2017-05-21 07:03:18,119.983796,31.847210,4,NaN,NaN,NaN


In [9]:
bbb = bbb.groupby((bbb.iloc[:,1].shift() != bbb.iloc[:,1]).cumsum())\
.filter(lambda x: len(x) <20)

In [99]:
bbb.loc[:,['LONGITUDE','LATITUDE']].to_csv('before_DCE_for_route11_Bus51938440.txt',header=None, index=None, sep=' ', mode='w')

In [10]:
bbb.to_csv('noise_filtered_Route11Bus51938440.csv',index=False)

### Noise filter by Table with only one direction
Since the result of Delay Coordinate Embedding(DEC) is not good as I expected to be, now I try to only use data from one direction to test its performance on DCE.

In [111]:
db_downstream = df[df['BUSSID']==4].loc[:,['ACTDATETIME','LONGITUDE','LATITUDE','BUSSID','STATIONTYPE','STATIONSEQNUM']] #select downstream data

In [113]:
db_downstream_filter = db_downstream.groupby((db_downstream.iloc[:,1].shift() != db_downstream.iloc[:,1]).cumsum())\
.filter(lambda x: len(x) <20)

In [114]:
db_downstream_filter.loc[:,['LONGITUDE','LATITUDE']].to_csv('before_DCE_for_route11_Bus51938440_downstream.txt',header=None, index=None, sep=' ', mode='w')